In [1]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer
from pystac_client import Client
import branca
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# Please use the collection name similar to the one used in STAC collection.

# Name of the collection for gosat budget methane. 
collection_name = "gosat-based-ch4budget-yeargrid-v1"

In [3]:
# Fetching the collection from STAC collections using appropriate endpoint.
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

{'id': 'gosat-based-ch4budget-yeargrid-v1',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/gosat-based-ch4budget-yeargrid-v1/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/gosat-based-ch4budget-yeargrid-v1'}],
 'title': 'GOSAT-based Top-down Total and Natural Methane Emissions v1',
 'extent': {'spatial': {'bbox': [[-180.0, -90.0, 180.0, 90.0]]},
  'temporal': {'interval': [['2019-01-01T00:00:00+00:00',
     '2019-12-31T00:00:00+00:00']]}},
 'license': 'CC-BY-4.0',
 'renders': {'dashboard': {'assets': ['post-total'],
   'nodata': 9.96921e+36,
   'rescale': [[0, 0.3]],
   'colormap_name': 'spectral_r'},
  'post-total

In [4]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [6]:
# Check total number of items available
number_of_items = get_item_count(collection_name)
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]
print(f"Found {len(items)} items")

Found 1 items


In [7]:
# Examining the first item in the collection
items[0]

{'id': 'gosat-based-ch4budget-yeargrid-v1-2019',
 'bbox': [-180.5, -90.5, 179.5, 89.5],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/gosat-based-ch4budget-yeargrid-v1'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/gosat-based-ch4budget-yeargrid-v1'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/gosat-based-ch4budget-yeargrid-v1/items/gosat-based-ch4budget-yeargrid-v1-2019'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/gosat-based-ch4budget-yeargrid-v1/items/gosat-based-ch4budget-yeargrid-v1-2019/map?assets=post-total&nodata=9.96921e%2B36&rescale=0%2C0.3&colormap_name=spectral_r',
   'rel': 'preview',
   'typ

In [8]:
# To access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
items = {item["properties"]["start_datetime"][:10]: item for item in items} 
asset_name = "prior-total"

In [9]:
# Fetching the min and max values for a specific item
rescale_values = {"max":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":items[list(items.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

In [10]:
items.keys()

dict_keys(['2019-01-01'])

In [11]:
color_map = "rainbow" # please select the color ramp from matplotlib library.
january_2019_tile = requests.get(
    f"{RASTER_API_URL}/collections/{items['2019-01-01']['collection']}/items/{items['2019-01-01']['id']}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
january_2019_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/gosat-based-ch4budget-yeargrid-v1/items/gosat-based-ch4budget-yeargrid-v1-2019/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=prior-total&color_formula=gamma+r+1.05&colormap_name=rainbow&rescale=0.0%2C2.121816635131836'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.5, -90.5, 179.5, 89.5],
 'center': [-0.5, -0.5, 0]}

In [12]:
# Set initial zoom and center of map for CH₄ Layer
# Centre of map [latitude,longitude]
map_ = folium.Map(location=(34, -118), zoom_start=6)

# January 2019
map_layer_2019 = TileLayer(
    tiles=january_2019_tile["tiles"][0],
    attr="GHG",
    opacity=0.7,
)
map_layer_2019.add_to(map_)

# # January 2012
# map_layer_2012 = TileLayer(
#     tiles=january_2012_tile["tiles"][0],
#     attr="GHG",
#     opacity=0.7,
# )
# map_layer_2012.add_to(map_.m2)

# visualising the map
map_

In [13]:
# Texas, USA
texas_aoi = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                # [13.686159004559698, -21.700046934333145],
                # [13.686159004559698, -23.241974326585833],
                # [14.753560168039911, -23.241974326585833],
                # [14.753560168039911, -21.700046934333145],
                # [13.686159004559698, -21.700046934333145],
                [-95, 29],
                [-95, 33],
                [-104, 33],
                [-104,29],
                [-95, 29]
            ]
        ],
        "type": "Polygon",
    },
}

In [14]:
# We will plug in the coordinates for a location inside the the polygon and a zoom level

aoi_map = Map(
    tiles="OpenStreetMap",
    location=[
        30,-100
    ],
    zoom_start=6,
)

folium.GeoJson(texas_aoi, name="Texas, USA").add_to(aoi_map)
aoi_map

In [15]:
# Check total number of items available
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name}/items?limit=300"
).json()["features"]
print(f"Found {len(items)} items")

Found 1 items


In [16]:
# Explore the first item
items[0]

{'id': 'gosat-based-ch4budget-yeargrid-v1-2019',
 'bbox': [-180.5, -90.5, 179.5, 89.5],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/gosat-based-ch4budget-yeargrid-v1'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/gosat-based-ch4budget-yeargrid-v1'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/gosat-based-ch4budget-yeargrid-v1/items/gosat-based-ch4budget-yeargrid-v1-2019'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/gosat-based-ch4budget-yeargrid-v1/items/gosat-based-ch4budget-yeargrid-v1-2019/map?assets=post-total&nodata=9.96921e%2B36&rescale=0%2C0.3&colormap_name=spectral_r',
   'rel': 'preview',
   'typ

In [17]:
# The bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
def generate_stats(item, geojson):
    result = requests.post(
        f"{RASTER_API_URL}/cog/statistics",
        params={"url": item["assets"][asset_name]["href"]},
        json=geojson,
    ).json()
    return {
        **result["properties"],
        "datetime": item["properties"]["start_datetime"][:10],
    }

In [18]:
%%time
stats = [generate_stats(item, texas_aoi) for item in items]

CPU times: user 15.7 ms, sys: 3.2 ms, total: 18.9 ms
Wall time: 636 ms


In [19]:
stats[0]

{'statistics': {'b1': {'min': 0.0002109968481818214,
   'max': 0.2379249483346939,
   'mean': 0.06609854102134705,
   'count': 36.0,
   'sum': 2.379547357559204,
   'std': 0.05244022027065174,
   'median': 0.06046847254037857,
   'majority': 0.0002109968481818214,
   'minority': 0.0002109968481818214,
   'unique': 50.0,
   'histogram': [[14.0, 7.0, 7.0, 10.0, 5.0, 3.0, 0.0, 1.0, 2.0, 1.0],
    [0.0002109968481818214,
     0.023982390761375427,
     0.04775378480553627,
     0.07152518630027771,
     0.09529657661914825,
     0.1190679669380188,
     0.14283937215805054,
     0.16661076247692108,
     0.19038215279579163,
     0.21415354311466217,
     0.2379249483346939]],
   'valid_percent': 100.0,
   'masked_pixels': 0.0,
   'valid_pixels': 50.0,
   'percentile_2': 0.002050149254500866,
   'percentile_98': 0.2056385725736618}},
 'datetime': '2019-01-01'}

In [20]:
def clean_stats(stats_json) -> pd.DataFrame:
    df = pd.json_normalize(stats_json)
    df.columns = [col.replace("statistics.b1.", "") for col in df.columns]
    df["date"] = pd.to_datetime(df["datetime"])
    return df


df = clean_stats(stats)
df.head(5)

datetime       min       max      mean  count       sum      std  \
0  2019-01-01  0.000211  0.237925  0.066099   36.0  2.379547  0.05244   

     median  majority  minority  unique  \
0  0.060468  0.000211  0.000211    50.0   

                                           histogram  valid_percent  \
0  [[14.0, 7.0, 7.0, 10.0, 5.0, 3.0, 0.0, 1.0, 2....          100.0   

   masked_pixels  valid_pixels  percentile_2  percentile_98       date  
0            0.0          50.0       0.00205       0.205639 2019-01-01

In [21]:
print(items[0]["properties"]["start_datetime"][:10])

2019-01-01


In [22]:
tile_2016 = requests.get(
    f"{RASTER_API_URL}/collections/{items[0]['collection']}/items/{items[0]['id']}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}",
).json()
tile_2016

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/gosat-based-ch4budget-yeargrid-v1/items/gosat-based-ch4budget-yeargrid-v1-2019/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=prior-total&color_formula=gamma+r+1.05&colormap_name=rainbow&rescale=0.0%2C2.121816635131836'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-180.5, -90.5, 179.5, 89.5],
 'center': [-0.5, -0.5, 0]}

In [23]:
# Use bbox initial zoom and map
# Set up a map located w/in event bounds

aoi_map_bbox = Map(
    tiles="OpenStreetMap",
    location=[
        30,-100
    ],
    zoom_start=8,
)

map_layer = TileLayer(
    tiles=tile_2016["tiles"][0],
    attr="GHG", opacity = 0.5
)

map_layer.add_to(aoi_map_bbox)

aoi_map_bbox